In [139]:
import sympy as sp
import numpy as np
import control as ct

In [127]:
# xii = sp.symbols("xii_{x:z}")
# vii = sp.symbols("vii_{x:z}")
# aii = sp.symbols("aii_{x:z}")
thrust, velocity = sp.symbols("T V")
g = sp.symbols("g")
mass = sp.symbols("m")
gamma = sp.Symbol("gamma")
gamma_dot = sp.Symbol(r"\dot{\gamma}")
theta = sp.Symbol("theta")
theta_dot = sp.Symbol(r"\dot{\theta}")

In [128]:
def t(angle):
    return sp.Matrix([[sp.cos(angle), -sp.sin(angle)],
                      [sp.sin(angle), sp.cos(angle)]])

## State space function

In [129]:
a_matrix = sp.Matrix([
    [0, 1, 0, 0],
    [thrust/velocity/mass, 0, thrust/velocity/mass, 0],
    [0, 0, 0, 1],
    [0, 0, 0, 0]
])

b_matrix = sp.Matrix([0, 0, 0, 1])

display(a_matrix)
display(b_matrix)

Matrix([
[      0, 1,       0, 0],
[T/(V*m), 0, T/(V*m), 0],
[      0, 0,       0, 1],
[      0, 0,       0, 0]])

Matrix([
[0],
[0],
[0],
[1]])

In [130]:
args = (thrust, mass, velocity)
a_matrix_func = sp.lambdify(args, a_matrix, modules='numpy')
b_matrix_func = sp.lambdify(args, b_matrix, modules='numpy')


def generate_state_space(thrust, mass, velocity):
    return a_matrix_func(thrust, mass, velocity),\
           b_matrix_func(thrust, mass, velocity),\
           np.eye(4), np.zeros((4, 1))

In [146]:
sys = ct.ss(*generate_state_space(1, 3, 4))

## Controller tuning

In [147]:
q_matrix = np.diag([1, 1, 1, 1])
r_matrix = np.diag([1])

In [150]:
gains, _, __ = ct.lqr(sys, q_matrix, r_matrix)
gains

matrix([[ 3.71181044, 12.42298034,  2.29759687,  2.36541619]])

In [155]:
# A-B*K
ss_fb = ct.ss(sys.A - sys.B @ gains, sys.B, sys.C, sys.D)